# Diseño de Prompts y Ejecución
1. Instalación `transformers`.
2. Importación de modelos desde Hugging Face (`roberta`, `bert`, `flan-t5`).
3. Carga de reseñas desde un archivo CSV.
4. Definición de tres técnicas de prompting: zero-shot, few-shot y chain-of-thought.
5. Generación de tres variantes de prompt por técnica.
6. Ejecución de los prompts en los modelos seleccionados.

In [1]:
%pip install transformers
%pip install pandas
%pip install tensorflow
%pip install torch
%pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [1]:
from transformers import pipeline
import pandas as pd

c:\Users\claud\OneDrive - U-tad\01_MAIS\6th Semester\02_BAIN\Practica4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("input/yelp_review.csv")
reviews = df["texto"].tolist()

In [3]:
models = {
    "alpaca": pipeline("text2text-generation", model="declare-lab/flan-alpaca-base", framework="pt"),
    "falcon": pipeline("text-generation", model="tiiuae/falcon-rw-1b", framework="pt"),
    "flan-t5": pipeline("text2text-generation", model="google/flan-t5-base")
}

Device set to use cpu
Device set to use cpu


Device set to use cpu


In [4]:
def zero_shot_prompt(text):
    return (
        "What is the sentiment of the following customer review? Respond with either Positive, Negative, or Neutral.\n"
        f"Review: {text}\n"
        "Sentiment:"
    )

def few_shot_prompt(text):
    return (
        "Analyze the review and decide if it's positive, negative or neutral, justifying your reasoning.\n\n"
        
        "Review: I’ve been using this phone for over a month now and I honestly couldn’t be happier. "
        "The performance is super fast — apps open instantly, and multitasking is smooth. The camera quality is outstanding, "
        "especially in low light conditions, and the battery easily lasts me a full day with heavy use. It also feels sturdy and well-built. "
        "Totally worth the price, highly recommended.\n"
        "Sentiment: positive\n\n"
        
        "Review: I had high hopes for this product, but I was extremely disappointed. "
        "After just one week of normal use, it completely stopped working. The screen froze constantly, and the battery would not hold a charge. "
        "I tried contacting customer service, but they never responded. Very poor quality for the price, and I regret buying it. "
        "Would not recommend this to anyone.\n"
        "Sentiment: negative\n\n"

        f"Review: {text}\n"
        "Sentiment:"
    )

def cot_prompt(text):
    return (
        "Analyze the review and decide if it's positive, negative or neutral, justifying your reasoning.\n\n"

        "Review: I’ve been using this phone for over a month now and I honestly couldn’t be happier. "
        "The performance is super fast — apps open instantly, and multitasking is smooth. The camera quality is outstanding"
        ", and the battery easily lasts me a full day with heavy use. "
        "Totally worth the price, highly recommended.\n"
        "Reasoning: The reviewer is clearly satisfied, using words like 'super fast', 'outstanding', and 'highly recommended'. "
        "These indicate a strong positive experience.\n"
        "Sentiment: positive\n\n"

        "Review: I had high hopes for this product, but I was extremely disappointed. "
        "After just one week of normal use, it completely stopped working."
        "I tried contacting customer service, but they never responded. Very poor quality for the price, and I regret buying it.\n"
        "Reasoning: The reviewer describes multiple failures and dissatisfaction, using phrases like 'extremely disappointed', 'poor quality', and 'regret buying it'.\n"
        "Sentiment: negative\n\n"

        f"Review: {text}\n"
        "Reasoning:"
    )


In [21]:
print("\n=== FLAN-T5 ===\n")

for text in reviews[:10]:
    prompt = few_shot_prompt(text)
    raw = models["flan-t5"](prompt, max_length=50, do_sample=False)[0]["generated_text"]
    result = raw.strip().split("\n")[0].strip()
    print(f"Review: {text}\nPrediction: {result}\n")



=== FLAN-T5 ===

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Prediction: positive

Review: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't 

In [22]:
print("\n=== ALPACA ===\n")
for text in reviews[:10]:
    prompt = cot_prompt(text)
    result = models["alpaca"](prompt, max_length=50, do_sample=False)[0]["generated_text"]
    print(f"Review: {text}\nPrediction: {result.strip()}\n")



=== ALPACA ===

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Prediction: Sentiment: positive

Review: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I j

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


Review: Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June.  He will be missed very much.  \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
Prediction: Sentiment: Negative

Review: I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you call the office, they'll put you through to a voice mail, that NO ONE ever answers or returns 

In [16]:
def normalize_falcon_sentiment(raw_output: str) -> str:
    lines = raw_output.strip().split("\n")
    sentiment_line = next((line for line in lines if "sentiment:" in line.lower()), "")
    
    if ":" in sentiment_line:
        sentiment = sentiment_line.split(":")[-1].strip().lower()
    else:
        sentiment = sentiment_line.strip().lower()

    # Be stricter about matching only when the model clearly labels sentiment
    if sentiment in ["strong buy"]:
        return "Positive"
    elif sentiment in ["dissatisfied", "strong sell"]:
        return "Negative"
    elif sentiment == "neutral":
        return "Neutral"
    else:
        return "Unknown"


In [17]:
print("\n=== FALCON ===\n")
for text in reviews[:10]:
    prompt = zero_shot_prompt(text)
    raw_result = models["falcon"](prompt, max_new_tokens=50, do_sample=False)[0]["generated_text"]
    
    # Show what Falcon actually said
    print(f"\nRAW RESULT:\n{raw_result.strip()}")
    
    sentiment = normalize_falcon_sentiment(raw_result)
    print(f"Review: {text}\nPrediction: {sentiment}\n")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== FALCON ===



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



RAW RESULT:
What is the sentiment of the following customer review? Respond with either Positive, Negative, or Neutral.
Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Sentiment: Strong Buy
I have been a patient of Dr. Goldberg for over 20 years. I have been a patient of Dr. Goldberg for over 20 years. I have been a patient of Dr. Goldberg for over 20 years. I have
Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; h

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


RAW RESULT:
What is the sentiment of the following customer review? Respond with either Positive, Negative, or Neutral.
Review: Dr. Eric Goldberg is a fantastic doctor who has correctly diagnosed every issue that my wife and I have had. Unlike many of my past doctors, Dr. Goldberg is very accessible and we have been able to schedule appointments with him and his staff very quickly. We are happy to have him in the neighborhood and look forward to being his patients for many years to come.
Sentiment: Strong Buy
I have been a patient of Dr. Goldberg for over 20 years. He is a very caring doctor and I have never had a bad experience with him. He is very thorough and listens to what you have to say. He is very
Review: Dr. Eric Goldberg is a fantastic doctor who has correctly diagnosed every issue that my wife and I have had. Unlike many of my past doctors, Dr. Goldberg is very accessible and we have been able to schedule appointments with him and his staff very quickly. We are happy to hav

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



RAW RESULT:
What is the sentiment of the following customer review? Respond with either Positive, Negative, or Neutral.
Review: I'm writing this review to give you a heads up before you see this Doctor. The office staff and administration are very unprofessional. I left a message with multiple people regarding my bill, and no one ever called me back. I had to hound them to get an answer about my bill. \n\nSecond, and most important, make sure your insurance is going to cover Dr. Goldberg's visits and blood work. He recommended to me that I get a physical, and he knew I was a student because I told him. I got the physical done. Later, I found out my health insurance doesn't pay for preventative visits. I received an $800.00 bill for the blood work. I can't pay for my bill because I'm a student and don't have any cash flow at this current time. I can't believe the Doctor wouldn't give me a heads up to make sure my insurance would cover work that wasn't necessary and was strictly prevent

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



RAW RESULT:
What is the sentiment of the following customer review? Respond with either Positive, Negative, or Neutral.
Review: Wing sauce is like water. Pretty much a lot of butter and some hot sauce (franks red hot maybe).  The whole wings are good size and crispy, but for $1 a wing the sauce could be better. The hot and extra hot are about the same flavor/heat.  The fish sandwich is good and is a large portion, sides are decent.
Sentiment: Strong Buy
I have been a fan of Wing Sauce for years. I have tried many of the flavors and they are all good. I have tried the original, the hot, the extra hot, the original hot, the original extra hot, the
Review: Wing sauce is like water. Pretty much a lot of butter and some hot sauce (franks red hot maybe).  The whole wings are good size and crispy, but for $1 a wing the sauce could be better. The hot and extra hot are about the same flavor/heat.  The fish sandwich is good and is a large portion, sides are decent.
Prediction: Positive


RAW RE